In [1]:
import os
import matplotlib.pyplot as plt
%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
/opt/conda/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
import torch
import sys
import os
import torch
import numpy
import importlib
import torch.nn.functional as F
from scipy.io import wavfile
from collections import defaultdict

from sklearn import manifold
from torchsummary import summary
import time

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

In [4]:
sys.path.append('/workspace/GREAT_ASV_system/train_dist')
sys.path.append('/workspace/GREAT_ASV_system/Model_exp')

# ResnetSE

In [5]:
SpeakerNetModel = importlib.import_module('models.'+'ResNetSE34V2').__getattribute__('MainModel')

In [6]:
# RESNET34SE
# (self, block, layers, num_filters, nOut, spec_aug, encoder_type='SAP', n_mels=40, log_input=True, **kwargs)
# model = ResNetSE(SEBasicBlock, [3, 4, 6, 3], num_filters, nOut, spec_aug, **kwargs)
S = SpeakerNetModel(n_mels=64, nOut=192, spec_aug=False, encoder_type='ASP')

ResNetSE34V2.py, Embedding size is 192, Encoder ASP, Spec_aug False.


In [7]:
model_path = '/workspace/LOGS_OUTPUT/tmp_logs/train_logs_201120/single_gpu_resnetsev2/model/model000000120.model'

In [8]:
# S.cuda()

In [9]:
loaded_state = torch.load(model_path, map_location="cpu")

In [10]:
self_state = S.state_dict()

for name, param in loaded_state['model'].items():
    origname = name
    if name not in self_state:
        name = name.replace("__S__.", "")

        if name not in self_state:
            print("#%s is not in the model."%origname)
            continue

    if self_state[name].size() != loaded_state['model'][origname].size():
        print("#Wrong parameter length: %s, model: %s, loaded: %s"%(origname, self_state[name].size(), loaded_state['model'][origname].size()))
        continue

    self_state[name].copy_(param)

#__L__.W is not in the model.


In [11]:
S.eval()

ResNetSE(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU(inplace=True)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): SEBasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=32, out_features=4, bias=True)
          (1): ReLU(inplace=True)
          (2): Linear(in_features=4, out_features=32, bias=True)
          (3): Sigmoid()
        )
      )
    )
    (1): SEBasicBlock(
      (c

In [12]:
def loadWAV(filename):

    # Maximum audio length
    
    max_audio = int(0*16000 + 240)

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)
    
    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]
    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat

In [13]:
test_list = '/workspace/DATASET/server9_ssd/voxceleb/vox_o_triallist.txt'
test_path = '/workspace/DATASET/server9_ssd/voxceleb'

In [14]:
files = []
with open(test_list) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split();

        ## Append random label if missing
        if len(data) == 2: data = [random.randint(0,1)] + data

        files.append(data[1])
        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [15]:
len(setfiles)

4708

In [17]:
starttime = time.time()
for count, wavline in enumerate(setfiles[:200]):
    wavline = os.path.join(test_path, wavline)
    raw_inp = loadWAV(wavline)
    raw_inp = torch.FloatTensor(raw_inp)
    
    ref_feat = S.forward(raw_inp).detach().cpu().numpy()
    
    print((count+1), end='\r')
endtime =time.time()
print((endtime-starttime)/200)

1.4298101341724396


In [18]:
S.cuda()

ResNetSE(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU(inplace=True)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): SEBasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=32, out_features=4, bias=True)
          (1): ReLU(inplace=True)
          (2): Linear(in_features=4, out_features=32, bias=True)
          (3): Sigmoid()
        )
      )
    )
    (1): SEBasicBlock(
      (c

In [19]:
summary(S, input_size=(120000,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       PreEmphasis-1               [-1, 120000]               0
       Spectrogram-2             [-1, 257, 751]               0
          MelScale-3              [-1, 64, 751]               0
    MelSpectrogram-4              [-1, 64, 751]               0
    InstanceNorm1d-5              [-1, 64, 751]               0
            Conv2d-6          [-1, 32, 64, 751]             320
              ReLU-7          [-1, 32, 64, 751]               0
       BatchNorm2d-8          [-1, 32, 64, 751]              64
            Conv2d-9          [-1, 32, 64, 751]           9,216
             ReLU-10          [-1, 32, 64, 751]               0
      BatchNorm2d-11          [-1, 32, 64, 751]              64
           Conv2d-12          [-1, 32, 64, 751]           9,216
      BatchNorm2d-13          [-1, 32, 64, 751]              64
AdaptiveAvgPool2d-14             [-1, 3

# ECAPA-TDNN1024

In [20]:
SpeakerNetModel = importlib.import_module('models.'+'EPACA-TDNN').__getattribute__('MainModel')

In [21]:
# RESNET34SE
# (self, block, layers, num_filters, nOut, spec_aug, encoder_type='SAP', n_mels=40, log_input=True, **kwargs)
# model = ResNetSE(SEBasicBlock, [3, 4, 6, 3], num_filters, nOut, spec_aug, **kwargs)
S = SpeakerNetModel(n_mels=40, nOut=192, spec_aug=False)

EPACA-TDNN.py, Embedding size is 192, Channels 1024, Spec_aug False.


In [22]:
model_path = '/workspace/LOGS_OUTPUT/tmp_logs/train_logs_201120/multi_gpu_epaca_tdnn/model/model000000060.model'

In [8]:
# S.cuda()

In [23]:
loaded_state = torch.load(model_path, map_location="cpu")

In [24]:
self_state = S.state_dict()

for name, param in loaded_state['model'].items():
    origname = name
    if name not in self_state:
        name = name.replace("__S__.", "")

        if name not in self_state:
            print("#%s is not in the model."%origname)
            continue

    if self_state[name].size() != loaded_state['model'][origname].size():
        print("#Wrong parameter length: %s, model: %s, loaded: %s"%(origname, self_state[name].size(), loaded_state['model'][origname].size()))
        continue

    self_state[name].copy_(param)

#__L__.W is not in the model.


In [25]:
S.eval()

ECAPA_TDNN(
  (instancenorm): InstanceNorm1d(40, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (torchfb): Sequential(
    (0): PreEmphasis()
    (1): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (layer1): Conv1dReluBn(
    (conv): Conv1d(40, 1024, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): Sequential(
    (0): Conv1dReluBn(
      (conv): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,), bias=False)
      (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Res2Conv1dReluBn(
      (convs): ModuleList(
        (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (2): Conv1d(128, 128, kernel_size=(3,), 

In [27]:
def loadWAV(filename):

    # Maximum audio length
    
    max_audio = int(0*16000 + 240)

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)
    
    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]
    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat

In [28]:
test_list = '/workspace/DATASET/server9_ssd/voxceleb/vox_o_triallist.txt'
test_path = '/workspace/DATASET/server9_ssd/voxceleb'

In [29]:
files = []
with open(test_list) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split();

        ## Append random label if missing
        if len(data) == 2: data = [random.randint(0,1)] + data

        files.append(data[1])
        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [30]:
len(setfiles)

4708

In [31]:
starttime = time.time()
for count, wavline in enumerate(setfiles[:200]):
    wavline = os.path.join(test_path, wavline)
    raw_inp = loadWAV(wavline)
    raw_inp = torch.FloatTensor(raw_inp)
    
    ref_feat = S.forward(raw_inp).detach().cpu().numpy()
    
    print((count+1), end='\r')
endtime =time.time()
print((endtime-starttime)/200)

1.3624333930015564


In [32]:
S.cuda()

ECAPA_TDNN(
  (instancenorm): InstanceNorm1d(40, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (torchfb): Sequential(
    (0): PreEmphasis()
    (1): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (layer1): Conv1dReluBn(
    (conv): Conv1d(40, 1024, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): Sequential(
    (0): Conv1dReluBn(
      (conv): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,), bias=False)
      (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Res2Conv1dReluBn(
      (convs): ModuleList(
        (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (2): Conv1d(128, 128, kernel_size=(3,), 

In [33]:
summary(S, input_size=(120000,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       PreEmphasis-1               [-1, 120000]               0
       Spectrogram-2             [-1, 257, 751]               0
          MelScale-3              [-1, 40, 751]               0
    MelSpectrogram-4              [-1, 40, 751]               0
    InstanceNorm1d-5              [-1, 40, 751]               0
            Conv1d-6            [-1, 1024, 751]         204,800
       BatchNorm1d-7            [-1, 1024, 751]           2,048
      Conv1dReluBn-8            [-1, 1024, 751]               0
            Conv1d-9            [-1, 1024, 751]       1,048,576
      BatchNorm1d-10            [-1, 1024, 751]           2,048
     Conv1dReluBn-11            [-1, 1024, 751]               0
           Conv1d-12             [-1, 128, 751]          49,152
      BatchNorm1d-13             [-1, 128, 751]             256
           Conv1d-14             [-1, 1

# ECAPA-TDNN512

In [34]:
SpeakerNetModel = importlib.import_module('models.'+'EPACA-TDNNL').__getattribute__('MainModel')

In [35]:
# RESNET34SE
# (self, block, layers, num_filters, nOut, spec_aug, encoder_type='SAP', n_mels=40, log_input=True, **kwargs)
# model = ResNetSE(SEBasicBlock, [3, 4, 6, 3], num_filters, nOut, spec_aug, **kwargs)
S = SpeakerNetModel(n_mels=40, nOut=192, spec_aug=False)

EPACA-TDNNL.py, Embedding size is 192, Channels 512, Spec_aug False.


In [36]:
model_path = '/workspace/LOGS_OUTPUT/tmp_logs/train_logs_201120/epaca_tdnnl_aldatest_beta0/model/model000000120.model'

In [8]:
# S.cuda()

In [37]:
loaded_state = torch.load(model_path, map_location="cpu")

In [38]:
self_state = S.state_dict()

for name, param in loaded_state['model'].items():
    origname = name
    if name not in self_state:
        name = name.replace("__S__.", "")

        if name not in self_state:
            print("#%s is not in the model."%origname)
            continue

    if self_state[name].size() != loaded_state['model'][origname].size():
        print("#Wrong parameter length: %s, model: %s, loaded: %s"%(origname, self_state[name].size(), loaded_state['model'][origname].size()))
        continue

    self_state[name].copy_(param)

#__L__.W is not in the model.
#DA_module.backbone.torchfb.0.flipped_filter is not in the model.
#DA_module.backbone.torchfb.1.spectrogram.window is not in the model.
#DA_module.backbone.torchfb.1.mel_scale.fb is not in the model.
#DA_module.backbone.layer1.conv.weight is not in the model.
#DA_module.backbone.layer1.bn.weight is not in the model.
#DA_module.backbone.layer1.bn.bias is not in the model.
#DA_module.backbone.layer1.bn.running_mean is not in the model.
#DA_module.backbone.layer1.bn.running_var is not in the model.
#DA_module.backbone.layer1.bn.num_batches_tracked is not in the model.
#DA_module.backbone.layer2.0.conv.weight is not in the model.
#DA_module.backbone.layer2.0.bn.weight is not in the model.
#DA_module.backbone.layer2.0.bn.bias is not in the model.
#DA_module.backbone.layer2.0.bn.running_mean is not in the model.
#DA_module.backbone.layer2.0.bn.running_var is not in the model.
#DA_module.backbone.layer2.0.bn.num_batches_tracked is not in the model.
#DA_module.bac

In [39]:
S.eval()

ECAPA_TDNN(
  (instancenorm): InstanceNorm1d(40, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (torchfb): Sequential(
    (0): PreEmphasis()
    (1): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (layer1): Conv1dReluBn(
    (conv): Conv1d(40, 512, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): Sequential(
    (0): Conv1dReluBn(
      (conv): Conv1d(512, 512, kernel_size=(1,), stride=(1,), bias=False)
      (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Res2Conv1dReluBn(
      (convs): ModuleList(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,)

In [40]:
def loadWAV(filename):

    # Maximum audio length
    
    max_audio = int(0*16000 + 240)

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)
    
    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]
    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat

In [41]:
test_list = '/workspace/DATASET/server9_ssd/voxceleb/vox_o_triallist.txt'
test_path = '/workspace/DATASET/server9_ssd/voxceleb'

In [42]:
files = []
with open(test_list) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split();

        ## Append random label if missing
        if len(data) == 2: data = [random.randint(0,1)] + data

        files.append(data[1])
        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [43]:
len(setfiles)

4708

In [44]:
starttime = time.time()
for count, wavline in enumerate(setfiles[:200]):
    wavline = os.path.join(test_path, wavline)
    raw_inp = loadWAV(wavline)
    raw_inp = torch.FloatTensor(raw_inp)
    
    ref_feat = S.forward(raw_inp).detach().cpu().numpy()
    
    print((count+1), end='\r')
endtime =time.time()
print((endtime-starttime)/200)

1.0166996669769288


In [45]:
S.cuda()

ECAPA_TDNN(
  (instancenorm): InstanceNorm1d(40, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (torchfb): Sequential(
    (0): PreEmphasis()
    (1): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (layer1): Conv1dReluBn(
    (conv): Conv1d(40, 512, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): Sequential(
    (0): Conv1dReluBn(
      (conv): Conv1d(512, 512, kernel_size=(1,), stride=(1,), bias=False)
      (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Res2Conv1dReluBn(
      (convs): ModuleList(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,)

In [46]:
summary(S, input_size=(120000,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       PreEmphasis-1               [-1, 120000]               0
       Spectrogram-2             [-1, 257, 751]               0
          MelScale-3              [-1, 40, 751]               0
    MelSpectrogram-4              [-1, 40, 751]               0
    InstanceNorm1d-5              [-1, 40, 751]               0
            Conv1d-6             [-1, 512, 751]         102,400
       BatchNorm1d-7             [-1, 512, 751]           1,024
      Conv1dReluBn-8             [-1, 512, 751]               0
            Conv1d-9             [-1, 512, 751]         262,144
      BatchNorm1d-10             [-1, 512, 751]           1,024
     Conv1dReluBn-11             [-1, 512, 751]               0
           Conv1d-12              [-1, 64, 751]          12,288
      BatchNorm1d-13              [-1, 64, 751]             128
           Conv1d-14              [-1, 

# X-vector

In [48]:
SpeakerNetModel = importlib.import_module('models.'+'X_vector').__getattribute__('MainModel')

In [49]:
# RESNET34SE
# (self, block, layers, num_filters, nOut, spec_aug, encoder_type='SAP', n_mels=40, log_input=True, **kwargs)
# model = ResNetSE(SEBasicBlock, [3, 4, 6, 3], num_filters, nOut, spec_aug, **kwargs)
S = SpeakerNetModel(n_mels=40, nOut=192, spec_aug=False)

X_vector.py, Embedding size is 192,  Spec_aug False.


In [50]:
model_path = '/workspace/LOGS_OUTPUT/tmp_logs/train_logs_201120/Xvecor_alda_0.0/model/model000000123.model'

In [8]:
# S.cuda()

In [51]:
loaded_state = torch.load(model_path, map_location="cpu")

In [52]:
self_state = S.state_dict()

for name, param in loaded_state['model'].items():
    origname = name
    if name not in self_state:
        name = name.replace("__S__.", "")

        if name not in self_state:
            print("#%s is not in the model."%origname)
            continue

    if self_state[name].size() != loaded_state['model'][origname].size():
        print("#Wrong parameter length: %s, model: %s, loaded: %s"%(origname, self_state[name].size(), loaded_state['model'][origname].size()))
        continue

    self_state[name].copy_(param)

#__L__.W is not in the model.
#DA_module.backbone.torchfb.0.flipped_filter is not in the model.
#DA_module.backbone.torchfb.1.spectrogram.window is not in the model.
#DA_module.backbone.torchfb.1.mel_scale.fb is not in the model.
#DA_module.backbone.tdnn1.0.weight is not in the model.
#DA_module.backbone.tdnn1.0.bias is not in the model.
#DA_module.backbone.tdnn1.2.weight is not in the model.
#DA_module.backbone.tdnn1.2.bias is not in the model.
#DA_module.backbone.tdnn1.2.running_mean is not in the model.
#DA_module.backbone.tdnn1.2.running_var is not in the model.
#DA_module.backbone.tdnn1.2.num_batches_tracked is not in the model.
#DA_module.backbone.tdnn2.0.weight is not in the model.
#DA_module.backbone.tdnn2.0.bias is not in the model.
#DA_module.backbone.tdnn2.2.weight is not in the model.
#DA_module.backbone.tdnn2.2.bias is not in the model.
#DA_module.backbone.tdnn2.2.running_mean is not in the model.
#DA_module.backbone.tdnn2.2.running_var is not in the model.
#DA_module.back

In [53]:
S.eval()

Xvector_1L(
  (instancenorm): InstanceNorm1d(40, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (torchfb): Sequential(
    (0): PreEmphasis()
    (1): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (tdnn1): Sequential(
    (0): Conv1d(40, 512, kernel_size=(5,), stride=(1,))
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tdnn2): Sequential(
    (0): Conv1d(512, 512, kernel_size=(3,), stride=(1,), dilation=(2,))
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tdnn3): Sequential(
    (0): Conv1d(512, 512, kernel_size=(3,), stride=(1,), dilation=(3,))
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tdnn4): Sequential(
    (0): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
    (1): ReLU(inplace=

In [54]:
def loadWAV(filename):

    # Maximum audio length
    
    max_audio = int(0*16000 + 240)

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)
    
    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]
    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat

In [55]:
test_list = '/workspace/DATASET/server9_ssd/voxceleb/vox_o_triallist.txt'
test_path = '/workspace/DATASET/server9_ssd/voxceleb'

In [56]:
files = []
with open(test_list) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split();

        ## Append random label if missing
        if len(data) == 2: data = [random.randint(0,1)] + data

        files.append(data[1])
        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [57]:
len(setfiles)

4708

In [58]:
starttime = time.time()
for count, wavline in enumerate(setfiles[:200]):
    wavline = os.path.join(test_path, wavline)
    raw_inp = loadWAV(wavline)
    raw_inp = torch.FloatTensor(raw_inp)
    
    ref_feat = S.forward(raw_inp).detach().cpu().numpy()
    
    print((count+1), end='\r')
endtime =time.time()
print((endtime-starttime)/200)

0.2526428437232971


In [59]:
S.cuda()

Xvector_1L(
  (instancenorm): InstanceNorm1d(40, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (torchfb): Sequential(
    (0): PreEmphasis()
    (1): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (tdnn1): Sequential(
    (0): Conv1d(40, 512, kernel_size=(5,), stride=(1,))
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tdnn2): Sequential(
    (0): Conv1d(512, 512, kernel_size=(3,), stride=(1,), dilation=(2,))
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tdnn3): Sequential(
    (0): Conv1d(512, 512, kernel_size=(3,), stride=(1,), dilation=(3,))
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tdnn4): Sequential(
    (0): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
    (1): ReLU(inplace=

In [60]:
summary(S, input_size=(120000,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       PreEmphasis-1               [-1, 120000]               0
       Spectrogram-2             [-1, 257, 751]               0
          MelScale-3              [-1, 40, 751]               0
    MelSpectrogram-4              [-1, 40, 751]               0
    InstanceNorm1d-5              [-1, 40, 751]               0
            Conv1d-6             [-1, 512, 747]         102,912
              ReLU-7             [-1, 512, 747]               0
       BatchNorm1d-8             [-1, 512, 747]           1,024
            Conv1d-9             [-1, 512, 743]         786,944
             ReLU-10             [-1, 512, 743]               0
      BatchNorm1d-11             [-1, 512, 743]           1,024
           Conv1d-12             [-1, 512, 737]         786,944
             ReLU-13             [-1, 512, 737]               0
      BatchNorm1d-14             [-1, 5

# ResnetSEL

In [61]:
SpeakerNetModel = importlib.import_module('models.'+'ResNetSE34L').__getattribute__('MainModel')

In [62]:
# RESNET34SE
# (self, block, layers, num_filters, nOut, spec_aug, encoder_type='SAP', n_mels=40, log_input=True, **kwargs)
# (self, block, layers, num_filters, nOut, encoder_type='SAP', n_mels=40, log_input=True, **kwargs)
# model = ResNetSE(SEBasicBlock, [3, 4, 6, 3], num_filters, nOut, spec_aug, **kwargs)
S = SpeakerNetModel(n_mels=64, nOut=192, encoder_type='ASP')

Embedding size is 192, encoder ASP.


In [7]:
# model_path = '/workspace/LOGS_OUTPUT/tmp_logs/train_logs_201120/single_gpu_resnetsev2/model/model000000120.model'

In [8]:
# S.cuda()

In [9]:
# loaded_state = torch.load(model_path, map_location="cpu")

In [10]:
# self_state = S.state_dict()

# for name, param in loaded_state['model'].items():
#     origname = name
#     if name not in self_state:
#         name = name.replace("__S__.", "")

#         if name not in self_state:
#             print("#%s is not in the model."%origname)
#             continue

#     if self_state[name].size() != loaded_state['model'][origname].size():
#         print("#Wrong parameter length: %s, model: %s, loaded: %s"%(origname, self_state[name].size(), loaded_state['model'][origname].size()))
#         continue

#     self_state[name].copy_(param)

#__L__.W is not in the model.


In [63]:
S.eval()

ResNetSE(
  (conv1): Conv2d(1, 16, kernel_size=(7, 7), stride=(2, 1), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): SEBasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=16, out_features=2, bias=True)
          (1): ReLU(inplace=True)
          (2): Linear(in_features=2, out_features=16, bias=True)
          (3): Sigmoid()
        )
      )
    )
    (1): SEBasicBlo

In [64]:
def loadWAV(filename):

    # Maximum audio length
    
    max_audio = int(0*16000 + 240)

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)
    
    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]
    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat

In [65]:
test_list = '/workspace/DATASET/server9_ssd/voxceleb/vox_o_triallist.txt'
test_path = '/workspace/DATASET/server9_ssd/voxceleb'

In [66]:
files = []
with open(test_list) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split();

        ## Append random label if missing
        if len(data) == 2: data = [random.randint(0,1)] + data

        files.append(data[1])
        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [67]:
len(setfiles)

4708

In [68]:
starttime = time.time()
for count, wavline in enumerate(setfiles[:200]):
    wavline = os.path.join(test_path, wavline)
    raw_inp = loadWAV(wavline)
    raw_inp = torch.FloatTensor(raw_inp)
    
    ref_feat = S.forward(raw_inp).detach().cpu().numpy()
    
    print((count+1), end='\r')
endtime =time.time()
print((endtime-starttime)/200)

1.0989412236213685


In [69]:
S.cuda()

ResNetSE(
  (conv1): Conv2d(1, 16, kernel_size=(7, 7), stride=(2, 1), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): SEBasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=16, out_features=2, bias=True)
          (1): ReLU(inplace=True)
          (2): Linear(in_features=2, out_features=16, bias=True)
          (3): Sigmoid()
        )
      )
    )
    (1): SEBasicBlo

In [70]:
summary(S, input_size=(120000,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       Spectrogram-1             [-1, 257, 751]               0
          MelScale-2              [-1, 64, 751]               0
    MelSpectrogram-3              [-1, 64, 751]               0
    InstanceNorm1d-4              [-1, 64, 751]               0
            Conv2d-5          [-1, 16, 32, 751]             784
       BatchNorm2d-6          [-1, 16, 32, 751]              32
              ReLU-7          [-1, 16, 32, 751]               0
            Conv2d-8          [-1, 16, 32, 751]           2,304
              ReLU-9          [-1, 16, 32, 751]               0
      BatchNorm2d-10          [-1, 16, 32, 751]              32
           Conv2d-11          [-1, 16, 32, 751]           2,304
      BatchNorm2d-12          [-1, 16, 32, 751]              32
AdaptiveAvgPool2d-13             [-1, 16, 1, 1]               0
           Linear-14                   